In [1]:
import os
import torch
import time
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from CNMP import CNMP 
from dataset import GridDataset 
import wandb
import math

In [2]:
# Hyperparameters
t_dim = 1                   # step index dimension
SM_dim = 32 ** 2            # grid flattened dimension (16*16)
encoder_hidden_dims = [1024, 1024, 1024]
latent_dim = 1024
decoder_hidden_dims = [1024, 1024, 1024]

batch_size = 15
num_epochs = 1000
learning_rate = 0.001

wandb.init(
    project="ped_forecasting",
    config={"learning_rate": 0.0001,
            "architecture": "CNMP",
            "epochs": num_epochs,
            "encoder_hidden_dims": encoder_hidden_dims,
            "latent_dim": latent_dim,
            "decoder_hidden_dims": decoder_hidden_dims}
)

wandb: Currently logged in as: arinc-demir (arinc-demir-bo-azi-i-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Load data generated earlier
data_path = "grids_tensor.pt"
grids_tensor = torch.load(data_path)

# Create the dataset and split into train and validation sets
dataset = GridDataset(grids_tensor, max_encodings=10, max_queries=10)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset = torch.utils.data.Subset(dataset, range(val_size, len(dataset)))
val_dataset = torch.utils.data.Subset(dataset, range(val_size))

# TODO I did this for trying to overfit the model with just one simulation.
#train_dataset = dataset
#val_dataset = dataset

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print(f"Device: {device}")

Device: cuda


/tmp/ipykernel_9892/490913078.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  grids_tensor = torch.load(data_path)


In [4]:

# Initialize the model and optimizer
model = CNMP(
    t_dim=t_dim,
    SM_dim=SM_dim,
    encoder_hidden_dims=encoder_hidden_dims,
    decoder_hidden_dims=decoder_hidden_dims,
    latent_dim=latent_dim
).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
start_time = time.time()
best_val_loss = math.inf  # Initialize best validation loss

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    epoch_train_loss = 0.0
    for padded_encodings, encodings_mask, padded_query_indices, padded_query_targets, queries_mask in train_loader:
        # Move data to GPU
        padded_encodings = padded_encodings.to(device)
        encodings_mask = encodings_mask.to(device)
        padded_query_indices = padded_query_indices.to(device)
        padded_query_targets = padded_query_targets.to(device)
        queries_mask = queries_mask.to(device)
        
        optimizer.zero_grad()

        output = model(padded_encodings, encodings_mask, padded_query_indices, queries_mask)
        loss = model.loss(output, padded_query_targets, queries_mask)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    avg_train_loss = epoch_train_loss / len(train_dataset)
    
    # Add validation every 100 epochs
    if (epoch + 1) % 100 == 0:
        model.eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for padded_encodings, encodings_mask, padded_query_indices, padded_query_targets, queries_mask in val_loader:
                padded_encodings = padded_encodings.to(device)
                encodings_mask = encodings_mask.to(device)
                padded_query_indices = padded_query_indices.to(device)
                padded_query_targets = padded_query_targets.to(device)
                queries_mask = queries_mask.to(device)
                output = model(padded_encodings, encodings_mask, padded_query_indices, queries_mask)
                loss = model.loss(output, padded_query_targets, queries_mask)
                epoch_val_loss += loss.item()
        avg_val_loss = epoch_val_loss / len(val_dataset)
        wandb.log({"validation_loss": avg_val_loss})
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "trained_model_best.pth")
            print(f"New best model found and saved with validation loss: {best_val_loss:.4f}")


    epoch_duration = time.time() - epoch_start_time
    elapsed_time = time.time() - start_time
    estimated_total_time = elapsed_time / (epoch + 1) * num_epochs
    estimated_time_left = estimated_total_time - elapsed_time
    wandb.log({"loss": avg_train_loss})
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f} Time Left: {estimated_time_left:.2f} seconds")

Epoch 1/1000, Training Loss: 0.1011 Time Left: 529.25 seconds
Epoch 2/1000, Training Loss: 0.1001 Time Left: 282.06 seconds
Epoch 3/1000, Training Loss: 0.0817 Time Left: 197.90 seconds
Epoch 4/1000, Training Loss: 0.0918 Time Left: 156.22 seconds
Epoch 5/1000, Training Loss: 0.0797 Time Left: 131.90 seconds
Epoch 6/1000, Training Loss: 0.0888 Time Left: 114.79 seconds
Epoch 7/1000, Training Loss: 0.0906 Time Left: 102.60 seconds
Epoch 8/1000, Training Loss: 0.0939 Time Left: 93.47 seconds
Epoch 9/1000, Training Loss: 0.0936 Time Left: 86.32 seconds
Epoch 10/1000, Training Loss: 0.0649 Time Left: 80.66 seconds
Epoch 11/1000, Training Loss: 0.1985 Time Left: 75.67 seconds
Epoch 12/1000, Training Loss: 0.0931 Time Left: 71.49 seconds
Epoch 13/1000, Training Loss: 0.0959 Time Left: 67.88 seconds
Epoch 14/1000, Training Loss: 0.0959 Time Left: 64.70 seconds
Epoch 15/1000, Training Loss: 0.0949 Time Left: 62.02 seconds
Epoch 16/1000, Training Loss: 0.0930 Time Left: 59.49 seconds
Epoch 17/1

In [ ]:
# Save the trained model
wandb.finish()
torch.save(model.state_dict(), "trained_model.pth")
print("Training complete. Model saved to trained_model.pth")
print(f"Best validation loss: {best_val_loss}")